# $\pi^+$

$$A_{Siv}^{\pi^{+}}(x,z,p_{hT}) = \mathcal{A}_0(z,p_{hT},M_1)\left( \frac{\mathcal{N}_u e_u^2 f_{u}(x)D_{\pi^+/u}(z) + {\mathcal{N}_{\bar{d}} e_{\bar{d}}^2 f_{\bar{d}}(x)D_{\pi^+/{\bar{d}}}(z)}}{e_u^2 f_{u}(x)D_{\pi^+/u}(z)+e_{\bar{d}}^2 f_{\bar{d}}(x)D_{\pi^+/\bar{d}}(z)}\right)$$

# $\pi^-$

## Functions

$$\mathcal{A}_0(z,p_{hT},M_1) = \frac{[z^2 \langle k_\perp^2 \rangle + \langle p_\perp^2 \rangle  ]\langle k_S^2 \rangle^2}{[z^2 \langle k_S^2 \rangle + \langle p_\perp^2 \rangle ]\langle k_\perp^2 \rangle^2} \exp \left[- \frac{p_{hT}^2z^2 \left(\langle k_S^2 \rangle - \langle k_\perp^2 \rangle \right)}{\left(z^2 \langle k_S^2 \rangle + \langle p_\perp^2 \rangle \right)\left(z^2 \langle k_\perp^2 \rangle + \langle p_\perp^2 \rangle  \right)} \right]\times \frac{\sqrt{2e}zp_{hT}}{M_1}$$

$$\mathcal{N}_q(x) = N_q x^{\alpha_q}(1-x)^{\beta_q}\frac{(\alpha_q + \beta_q)^{(\alpha_q + \beta_q)}}{\alpha_q^{\alpha_q}\beta_q^{\beta_q}}$$

$f_{u}(x)$ is the lhapdf

$D_{\pi^+/u}(z)$ is the fragmentation function

## Constants

- $\langle k_\perp^2 \rangle$ 
- $\langle p_\perp^2 \rangle$
- $\langle k_S^2 \rangle$
- $e_u$ = $2/3$
- $e_\bar{u}$ = $-2/3$
- $e_d$ = $-1/3$
- $e_\bar{d}$ = $1/3$
- $e_s$ = $-1/3$
- $e_\bar{s}$ = $1/3$
- $e$ = $1/137$

## Unknown parameters

- $N_u$
- $\alpha_u$
- $\beta_u$
- $N_{\bar{d}}$
- $\alpha_{\bar{d}}$
- $\beta_{\bar{d}}$
- $M_1$

$$\mathcal{N}_q(x) = N_q x^{\alpha_q}(1-x)^{\beta_q}\frac{(\alpha_q + \beta_q)^{(\alpha_q + \beta_q)}}{\alpha_q^{\alpha_q}\beta_q^{\beta_q}}$$

In [31]:
import lhapdf

In [48]:
class Hadron(object):
    def __init__(self, kperp2avg=.57, pperp2avg=.12, pdfset='JAM19PDF_proton_nlo',
                 ff_pion='JAM19FF_pion_nlo', ff_kaon='JAM19FF_kaon_nlo', QQ=10):
        
        self.pdfData = lhapdf.mkPDF(pdfset)
        self.ffData = lhapdf.mkPDF(ff_pion, 0)
        # needs to be extended to generalize for kaons
        self.QQ = QQ
        self.kperp2avg = kperp2avg
        self.pperp2avg = pperp2avg
        self.eu = 2/3
        self.eubar = -2/3
        self.ed = -1/3
        self.edbar = 1/3
        self.es = -1/3
        self.esbar = 1/3
        self.e = 1/137
    

    def pdf(self, flavor, x):
        return np.array([self.pdfData.xfxQ2(flavor, ax, self.QQ) for ax in x])
    
    def ff(self, flavor, z):
        return np.array([self.ffData.xfxQ2(flavor, az, self.QQ) for az in z])
    
    
    def A0(self, z, pht, m1):
        ks2avg = self.kperp2avg*m1**2/(m1**2 + self.kperp2avg)
        topfirst = (np.square(z)*self.kperp2avg + self.pperp2avg)*self.kperp2avg**2
        bottomfirst = (np.square(z)*ks2avg + self.pperp2avg) * self.kperp2avg**2
        exptop = np.square(pht) * np.square(z) * (ks2avg - self.kperp2avg)
        expbottom = (np.square(z)*ks2avg + self.pperp2avg) * (np.square(z)*self.kperp2avg + self.pperp2avg)
        last = np.sqrt(2*self.e) * z * pht / m1
        
        return (topfirst/bottomfirst) * np.exp(-exptop/expbottom) * last
    
    
    def NN(self, x, n, a, b):
        return n*np.power(x, a) * np.power(1 - x, b) * (((a + b)**(a + b))/(a**a * b**b))

    def NNanti(self, x, n):
        return n
            
    
    

In [49]:
import numpy as np

In [50]:
class PiPlus(Hadron):
    def __init__(self, kperp2avg=.57, pperp2avg=.12, pdfset='JAM19PDF_proton_nlo',
                 ff_pion='JAM19FF_pion_nlo', QQ=10):
        
        super().__init__(kperp2avg=kperp2avg, pperp2avg=pperp2avg, pdfset=pdfset,
                 ff_pion=ff_pion, QQ=QQ)
        
    
    def sivers(self, kins, Nu, au, bu, Ndbar, adbar, bdbar, m1):
        x = kins[:, 0]
        z = kins[:, 1]
        pht = kins[:, 2]
        a0 = self.A0(z, pht, m1)
        topleft = self.NN(x, Nu, au, bu) * self.eu**2 * self.pdf(2, x) * self.ff(2, z)
        topright = self.NNanti(x, Ndbar) * self.edbar**2 * self.pdf(-1, x) * self.ff(-1, z)
        bottomleft = self.eu**2 * self.pdf(2, x) * self.ff(2, z)
        bottomright = self.edbar**2 * self.pdf(-1, x) * self.ff(-1, z)
        return a0*((topleft + topright)/(bottomleft + bottomright))

In [51]:
pip = PiPlus()

In [38]:
np.array([1, 2, 3])**2

array([1, 4, 9])

In [39]:
import pandas as pd

In [40]:
df = pd.read_csv('COMPASS_d_2009.csv')

In [41]:
pipdata = df.loc[df['hadron'] == 'pi+', :]

In [43]:
kins = np.array(pipdata[['x', 'z', 'phT']])

In [ ]:
kins

In [52]:
pip.sivers(kins, 1., 1., 1., 1., 1., 1., 1.)

array([0.00307976, 0.00429643, 0.0059507 , 0.00748906, 0.01047004,
       0.01519049, 0.01977303, 0.0198893 , 0.00453345, 0.00530429,
       0.00543865, 0.00576814, 0.00688007, 0.00857435, 0.01127992,
       0.01638525, 0.02574682, 0.0018364 , 0.00305678, 0.00440506,
       0.00601993, 0.0082816 , 0.01175141, 0.01877372, 0.03707232,
       0.19386578])

In [28]:
1

0

# Function testing

## A0

### Ishara

In [10]:
import numpy as np
import pandas as pd

In [13]:
def ks2Avg(m1,kperp2Avg):
    test_ks2Avg=((m1**2)*kperp2Avg)/((m1**2)+kperp2Avg)
    return test_ks2Avg

def A0(z,pht,m1,kperp2Avg,pperp2Avg,eCharg):
    tempA0part1=(((z**2)*kperp2Avg+pperp2Avg)*((ks2Avg(m1,kperp2Avg))**2))/((((z**2)*(ks2Avg(m1,kperp2Avg))+pperp2Avg))*kperp2Avg**2)
    tempA0part21=(pht**2)*(z**2)*(ks2Avg(m1,kperp2Avg) - kperp2Avg)
    tempA0part22=((z**2)*(ks2Avg(m1,kperp2Avg))+pperp2Avg)*((z**2)*kperp2Avg+pperp2Avg)
    tempA0part2=np.exp(-tempA0part21/tempA0part22)
    tempA0part3=(np.sqrt(2*eCharg))*z*pht/m1
    tempA0=tempA0part1*tempA0part2*tempA0part3
    return tempA0

In [14]:
print(A0(0.2, 0.3, 1, 2.5, 1.5, 1/137))

0.0006210773138177469


### Nick

In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
import functions

In [8]:
had = functions.Hadron(kperp2avg=2.5, pperp2avg=1.5)

In [9]:
print(had.A0(.2, .3, 1))

0.0006210773138177469


## NN

### Ishara

In [16]:
def NNu(x,Nu,alphau,betau):
    tempNNu = Nu*(x**alphau)*((1-x)**(betau))*((alphau+betau)**(alphau+betau))/((alphau**alphau)*(betau**betau))
    return tempNNu

def NNubar(x,Nubar,alphaubar=0,betaubar=0):
    #tempNNubar = Nubar*(x**alphaubar)*((1-x)**(betaubar))*((alphaubar+betaubar)**(alphaubar+betaubar))/((alphaubar**alphaubar)*(betaubar**betaubar))
    tempNNubar = Nubar
    return tempNNubar

def NNd(x,Nd,alphad,betad):
    tempNNd = Nd*(x**alphad)*((1-x)**(betad))*((alphad+betad)**(alphad+betad))/((alphad**alphad)*(betad**betad))
    return tempNNd

def NNdbar(x,Ndbar,alphadbar=0,betadbar=0):
    #tempNNdbar = Ndbar*(x**alphadbar)*((1-x)**(betadbar))*((alphadbar+betadbar)**(alphadbar+betadbar))/((alphadbar**alphadbar)*(betadbar**betadbar))
    tempNNdbar = Ndbar
    return tempNNdbar

def NNs(x,Ns,alphas,betas):
    #tempNNu = Ns*(x**alphas)*((1-x)**(betas))*((alphas+betas)**(alphas+betas))/((alphas**alphas)*(betas**betas))
    tempNNs = 0
    return tempNNs

def NNsbar(x,Nsbar,alphasbar,betasbar):
    #tempNNsbar = Nsbar*(x**alphasbar)*((1-x)**(betasbar))*((alphasbar+betasbar)**(alphasbar+betasbar))/((alphasbar**alphasbar)*(betasbar**betasbar))
    tempNNsbar = 0
    return tempNNsbar

In [23]:
NNu(2, 2, 2, 2)

128.0

### Nick

In [40]:
dic = {'a': 1, 'b':2, 'c':3}

In [42]:
[dic[x] for x in ['a', 'b']]

[1, 2]

In [24]:
print(had.NN(2, 2, 2, 2))

128.0


## PiPlus

### Ishara

In [37]:
import lhapdf

def xFxQ2(dataset,flavor,x,QQ):
    temp_parton_dist_x=(dataset.xfxQ2(flavor, x, QQ))
    return temp_parton_dist_x

def zFzQ(dataset,flavor,zz,QQ):
    # Here "0" represents the central values from the girds
    temp_zD1=lhapdf.mkPDF(dataset[0], 0)
    zD1_vec=(temp_zD1.xfxQ2(flavor,zz,QQ))
    return zD1_vec



def ASivPiP(x,z,pht,m1,kperp2Avg,pperpAvg,eCharg,eu,edbar,Nu,alphau,betau,Ndbar,QQ):
    Ucontribution1 = NNu(x,Nu,alphau,betau)*(eu**2)*xFxQ2(PDFdataset,2,x,QQ)*zFzQ(FF_pion_dataset,2,z,QQ)
    dbarcontribution1 = NNdbar(x,Ndbar)*(edbar**2)*xFxQ2(PDFdataset,-1,x,QQ)*zFzQ(FF_pion_dataset,-1,z,QQ)
    Ucontribution2 = (eu**2)*xFxQ2(PDFdataset,2,x,QQ)*zFzQ(FF_pion_dataset,2,z,QQ)
    dbarcontribution2 = (edbar**2)*xFxQ2(PDFdataset,-1,x,QQ)*zFzQ(FF_pion_dataset,-1,z,QQ)
    tempNumerator = Ucontribution1 + dbarcontribution1
    tempDenominator = Ucontribution2 + dbarcontribution2
    tempASivPiP = A0(z,pht,m1,kperp2Avg,pperpAvg,eCharg)*tempNumerator/tempDenominator
    return tempASivPiP

Datafile='Data/HERMES_p_2009.csv'
PDFdataset = lhapdf.mkPDF("JAM19PDF_proton_nlo")
FF_pion_dataset=["JAM19FF_pion_nlo"]

M1=np.sqrt(0.8)
Kp2A=0.57
Pp2A=0.12
ee=1/137
eU=2/3
eDbar=1/3
AlphaU=1.0
BetaU=6.6
NU=0.18
NDbar=-0.06


ASivPiP(0.9,0.234,0.346,M1,Kp2A,Pp2A,ee,eU,eDbar,NU,AlphaU,BetaU,NDbar,2.423)

-1.0623623445743228e-07

### Nick

In [31]:
pip = functions.PiPlus()

In [38]:
kins = np.array([[0.9,0.234,0.346, 2.423]])

In [39]:
pip.sivers(kins, 0.18, 1., 6.6, -.06, np.sqrt(.8))

array([-1.06236234e-07])